<h2>Objetivo: detectar qué productos son los más comprados por ciertos grupos etarios.</h2>
<h3>Rangos de edad que vamos a evaluar: 18-24, 25-35, 36-49, 50+</h3>

<h4>1.Fase de Análisis exploratorio</h4>

In [406]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [410]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/micaelakorol21/datasets/refs/heads/main/customer_data.csv')

In [ ]:
df_customer.shape

In [ ]:
df_customer.isna().sum()

In [ ]:
df_customer.dtypes

In [ ]:
# Observamos el promedio de ingreso:
df_customer['income'].describe()

In [ ]:
# Importe de la compra promedio entre hombres y mujeres:
df_customer.groupby("gender")["purchase_amount"].mean()

In [ ]:
# Categorias mas vendidas
df_customer.groupby("product_category")["purchase_amount"].mean()

In [ ]:
# Cuantas categorias hay:
df_customer['product_category'].value_counts()

In [ ]:
sns.set(style="whitegrid")

# Boxplot de edad
plt.figure(figsize=(8, 4))
sns.boxplot(x=df_customer["age"], color="skyblue")
plt.title("Boxplot de Edad")
plt.xlabel("Edad")
plt.show()

In [ ]:
df_customer['age'].describe()

In [ ]:
# Eliminamos los menores de 18 años, ya que nuestro análisis se centra > 18 hasta 99 años.

In [ ]:
df_customer = df_customer[df_customer['age'] >= 18]

In [ ]:
sns.boxplot(x=df_customer["age"], color="orange")
plt.title("Boxplot de Edad (filtrando menores)")
plt.show()

<h4>Analizamos la columna gender: </h4>

In [ ]:
# Cuántos registros hay de cada género:
df_customer['gender'].value_counts()

In [ ]:
# Porcentaje de representación de cada género:
df_customer['gender'].value_counts(normalize=True) * 100

In [ ]:
# Gasto promedio por género:
df_customer.groupby('gender')['purchase_amount'].mean()

In [ ]:
sns.boxplot(data=df_customer, x='gender', y='purchase_amount')

<p>El 25% de los compradores, gastó $6000</p>
<p>El 50% de los compradores, gastó 9500</p>
<p>El otro 25% gastó 13.500 aprox. El 75% de los compradores, gastó menos que Q3.</p>

In [ ]:
# Frecuencia de compra por género
df_customer.groupby('gender')['purchase_frequency'].value_counts(normalize=True)

In [ ]:
# Preferencias de categoria 
df_customer.groupby('gender')['product_category'].value_counts()

In [ ]:
# Calculamos la frecuencia absoluta y la proporción de categorías de productos para un género específico con una función reutilziable.
def get_gender_product_proportions(df_customer, gender):
    # Cuenta cuántas veces aparece cada combinación de género y categoría.
    counts = df_customer.groupby(['gender', 'product_category']).size().reset_index(name='count')
    # Total de compras para cada género
    totals = counts.groupby('gender')['count'].transform('sum')
    # Proporción de cada categoría dentro de un género
    counts['proportion'] = counts['count'] / totals

    gender_counts = counts[counts['gender'] == gender]

    return gender_counts

In [ ]:
female_data = get_gender_product_proportions(df_customer, 'Female')
print(female_data.head())

<p>Las mujeres gastan más en categorias como: Electrónica 30%, Ropa 20% y comida 14%.</p>
<p>Además, el 4.92% de las compras de mujeres fueron de productos de belleza y 14% en compras de libros.</p>

In [ ]:
male_data = get_gender_product_proportions(df_customer, 'Male')
print(male_data.head())

<p>Los hombres gastan más en categorias como: Electrónica, Ropa, Libros y Comida.</p>

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(data=counts, x='product_category', y='proportion', hue='gender', palette='pastel')
plt.title('Preferencias de categoría por género')
plt.ylabel('Proporción de compras')
plt.xticks(rotation=45)
plt.show()

In [ ]:
pivot_df = counts.pivot(index='product_category', columns='gender', values='proportion')

pivot_df.plot(kind='bar', stacked=True, figsize=(12,6), colormap='Pastel1')
plt.title('Preferencias de categoría por género (barras apiladas)')
plt.ylabel('Proporción de compras')
plt.xticks(rotation=45)
plt.show()

<b>La distribución y preferencias de productos entre hombres y mujeres son muy similares, lo que coincide con que ambos géneros están representados casi por igual en la muestra.</b>

In [ ]:
df_customer.columns

<h4>Análisis de la columna: 'region'</h4>

In [ ]:
df_customer['region'].value_counts()

In [ ]:
df_customer['region'].describe()

<h4>Análisis de la columna: 'income'</h4>

In [ ]:
plt.figure(figsize=(8, 2))
sns.boxplot(x=df_customer['income'], color='lightgreen')
plt.title('Boxplot de Ingresos')
plt.xlabel('Ingreso')
plt.show()

<b>Los valores están razonablemente distribuidos y no hay ingresos extremos que puedan distorsionar el análisis </b>

In [ ]:
df_customer.columns

<h4>Análisis de la columna "education" </h4>

In [ ]:
df_customer['education'].value_counts()

In [ ]:
df_customer['education'].describe()

<h4>Análisis de la columna "loyalty_status" </h4>

In [ ]:
df_customer['loyalty_status'].dtype

In [ ]:
df_customer['loyalty_status'].value_counts()

In [ ]:
df_customer['loyalty_status'].describe()

In [ ]:
df_customer['loyalty_status'].value_counts().plot(kind='bar', color='pink')
plt.title('Distribución de Loyalty Status')
plt.xlabel('Loyalty Status')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.show()

<p>Esto podría indicar que, aunque se tiene un número considerable de clientes leales (como los de Gold o Silver), la mayoría de tus clientes no alcanzan un nivel de lealtad muy alto, sino que están en un punto medio Regular.</p>

<h4>Análisis de la columna "purchase_frequency" </h4>

In [ ]:
df_customer['purchase_frequency'].dtype

In [ ]:
df_customer['purchase_frequency'].value_counts() 

In [ ]:
# Datos
frecuencia = {
    'rare': 49885,
    'occasional': 29805,
    'frequent': 20042
}

# Total de registros
total = sum(frecuencia.values())

# Calcular proporciones
proporciones = {key: value / total for key, value in frecuencia.items()}
print(proporciones)

In [ ]:
import matplotlib.pyplot as plt

# Datos para gráfico
categorias = list(frecuencia.keys())
valores = list(frecuencia.values())

# Crear gráfico
plt.figure(figsize=(8, 6))
plt.bar(categorias, valores, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Distribución de Frecuencia de Compra')
plt.xlabel('Frecuencia de Compra')
plt.ylabel('Número de Clientes')
plt.xticks(rotation=45)
plt.show()

<p> A partir de la distribución de la frecuencia de compra, donde el 50% de los clientes compran raramente, el 30% lo hacen de manera ocasional y solo el 20% de manera frecuente, se observa una baja fidelidad entre los clientes. Esto indica que una gran parte de la base de clientes no tiene un comportamiento de compra regular.</p>

<h4>Análisis de la columna: "purchase_amount": </h4>

In [ ]:
plt.figure(figsize=(8, 2))
sns.boxplot(x=df_customer['purchase_amount'], color='pink')
plt.title('Boxplot de "purchase_amount"')
plt.xlabel('Monto de compra')
plt.show()

<b>En el análisis de los montos de compra, observamos algunos valores atípicos (outliers) que representan compras mucho más altas que el promedio. Estos valores son representados por los círculos fuera del rango en el boxplot. Aunque estos puntos se consideren técnicamente outliers, no los eliminaremos del análisis, ya que representan clientes de alto gasto y pueden ser fundamentales para entender ciertos patrones de consumo.</b>

<h4>Análisis de la columna: "product_category": </h4>

In [ ]:
print(df_customer['product_category'].dtype)

In [ ]:
category_counts = df_customer['product_category'].value_counts()
# Gráfico de barras para visualizar la distribución
plt.figure(figsize=(10, 6))
category_counts.plot(kind='bar', color='purple')
plt.title('Distribución de Categorías de Producto')
plt.xlabel('Categoría de Producto')
plt.ylabel('Número de Compras')
plt.xticks(rotation=45)
plt.show()

<h4>Análisis de la columna: "promotion_usage": </h4>

In [ ]:
# Distribución de uso de promociones
promotion_usage_counts = df_customer['promotion_usage'].value_counts()

# Gráfico de barras
plt.figure(figsize=(8, 6))
promotion_usage_counts.plot(kind='bar', color='orange')
plt.title('Distribución de Uso de Promociones')
plt.xlabel('Uso de Promoción')
plt.ylabel('Número de Clientes')
plt.xticks(rotation=0)
plt.show()
# 0 indica que el cliente no usa promociones
# 1 indica que el cliente usa promociones

<b>Podemos observar que la mayoria de los clientes no aprovechan las promociones.</b>

<h4>Análisis de la columna: "satisfaction_score"</h4>

In [ ]:
print(df_customer['satisfaction_score'].dtype)

In [ ]:
# Ver los valores únicos de la columna 'satisfaction_score'
print(df_customer['satisfaction_score'].unique())

In [ ]:
df_customer['satisfaction_score'].describe()

In [ ]:
# Histograma de Rango de satisfacción de 0 a 10.
plt.figure(figsize=(8, 6))
df_customer['satisfaction_score'].plot(kind='hist', bins=10, color='salmon', edgecolor='black')
plt.title('Distribución de Satisfacción del Cliente')
plt.xlabel('Puntuación de Satisfacción')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
plt.figure(figsize=(8, 2))
sns.boxplot(x=df_customer['satisfaction_score'], color='orange')
plt.title('Boxplot de "satisfaction_score"')
plt.xlabel('satisfaction_score')
plt.show()

<b>Al observar la distribución de los puntajes de satisfacción, podemos ver que la mayoría de los clientes se encuentran en los valores centrales de la escala, con una ligera inclinación hacia la satisfacción (valores cercanos a 5 y 6). Esto indica que muchos clientes se sienten satisfechos o neutros con su experiencia.</b> <br>
<b>En el boxplot de la distribución de los puntajes de satisfacción, podemos observar que los valores de 0 y 10 están fuera del rango intercuartílico y se identifican como outliers. Sin embargo, estos valores no serán eliminados, ya que representan extremos válidos de la escala: 0  muy insatisfecho, 10 muy satisfecho.</b>

<h3>Mapa de correlacion entre variables: Muestra qué variables están más relacionadas entre sí.</h3>

In [ ]:
sns.heatmap(df_customer.corr(numeric_only=True), annot=True, cmap="coolwarm")

<b>Existe una relación significativa entre el ingreso de una persona y su gasto. Además, se observa una fuerte correlación entre la edad y la puntuación de satisfacción. Por otro lado, el uso de promociones muestra una relación moderada con el nivel de ingreso</b>

<h3>Discretización de la variable 'age'</h3>

In [ ]:
bins = [18, 24, 35, 49, 100] 
labels = ['18-24', '25-35', '36-49', '50+']

# Crear la columna 'age_group' con los grupos etarios definidos
df_customer['age_group'] = pd.cut(df_customer['age'], bins=bins, labels=labels, right=False)

# Verificar los primeros registros con la nueva columna 'age_group'
df_customer[['age', 'age_group']].head()

In [ ]:
# Agrupamos y calculamos
counts_age_group = df_customer.groupby(['age_group', 'product_category']).size().reset_index(name='count')
totals_age_group = counts_age_group.groupby('age_group')['count'].transform('sum')
counts_age_group['proportion'] = counts_age_group['count'] / totals_age_group

print(counts_age_group)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=counts_age_group, x='age_group', y='count', hue='product_category')

plt.title('Distribución de categorías de productos por grupo etario')
plt.xlabel('Grupo Etario')
plt.ylabel('Frecuencia de compras')
plt.xticks(rotation=45)
plt.legend(title='Categoría de Producto')

plt.show()

<h4>Conclusiones y Análisis final:</h4>

<b>Grupo etario: 18 - 24 años</b>
<p>La categoría más comprada por este grupo es Electrónica, seguida de Ropa.</p>
<p>Beauty (cosméticos) es la categoría con la menor preferencia en este grupo etario.</p>

<b>Grupo etario: 25 - 35 años</b>
<p>Al igual que el grupo anterior, Electrónica es la categoría más comprada, seguida de Ropa.</p>
<p>Beauty sigue siendo la categoría menos elegida en este grupo, aunque ha experimentado un pequeño aumento en comparación con el grupo anterior.</p>

<b>Grupo etario: 36 - 49 años</b>
<p>Electrónica es la categoría con mayor compra, seguida de Ropa.</p>
<p>Home es la categoría menos comprada, y se ha visto una disminución de compra en comparación con el grupo etario anterior.</p>

<b>Grupo Etario: 50+ años</b>
<p>Este grupo mostró pocos datos representativos, con solo una compra en Food.</p>

<ul>
<li>Electrónica es la categoría más comprada en todos los grupos etarios, mostrando una fuerte preferencia por productos tecnológicos a lo largo de las edades.</li>
<li>Ropa sigue siendo una categoría relevante, especialmente entre los grupos de menor edad (18-24 y 25-35), donde se encuentra en la segunda posición en cuanto a compras.</li>
<li>Por otro lado, Beauty (cosméticos) y Home tienen una baja preferencia generalizada, apareciendo en los últimos lugares en términos de compra en todos los grupos etarios.</li>
</ul>

<p>Aunque el objetivo principal de este análisis era detectar qué productos son los más comprados por ciertos grupos etarios, se optó por explorar todas las columnas del dataset en primer lugar. Esta estrategia permitió obtener una comprensión más profunda de los datos y sus relaciones, lo cual no solo aporta valor al análisis actual, sino que también abre la puerta a futuros estudios.</p>